Example 1 : Two dataframes with reconciliation column key containing Matched/Mismatched elements, with reports

In [107]:
import pandas as pd

source_a = pd.DataFrame({
    'Transaction_ID': ['TXN001', 'TXN002', 'TXN003', 'TXN004'],
    'Account_Number': [12345, 12346, 12347, 12348],
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'Amount': [500.00, 1500.00, 750.00, 300.00]
})

source_b = pd.DataFrame({
    'Transaction_ID': ['TXN005', 'TXN006', 'TXN007', 'TXN008'],
    'Account_Number': [12345, 12346, 12349, 12347],
    'Date': ['2023-01-01', '2023-01-02', '2023-01-05', '2023-01-03'],
    'Amount': [500.00, 1500.00, 100.00, 700.00]
})

display(source_a)
display(source_b)


,Transaction_ID,Account_Number,Date,Amount
0,TXN001,12345,2023-01-01,500.0
1,TXN002,12346,2023-01-02,1500.0
2,TXN003,12347,2023-01-03,750.0
3,TXN004,12348,2023-01-04,300.0


,Transaction_ID,Account_Number,Date,Amount
0,TXN005,12345,2023-01-01,500.0
1,TXN006,12346,2023-01-02,1500.0
2,TXN007,12349,2023-01-05,100.0
3,TXN008,12347,2023-01-03,700.0


Mismatches

Solution A

In [66]:
reconciled = pd.merge(source_a, source_b, on='Account_Number', how='outer', suffixes=('_SourceA', '_SourceB'), indicator=True)

mismatches = reconciled[reconciled['_merge'] != 'both']
display(mismatches)

,Transaction_ID_SourceA,Account_Number,Date_SourceA,Amount_SourceA,Transaction_ID_SourceB,Date_SourceB,Amount_SourceB,_merge
3,TXN004,12348,2023-01-04,300.0,NaN,NaN,NaN,left_only
4,NaN,12349,NaN,NaN,TXN007,2023-01-05,100.0,right_only


Solution B

In [108]:
pd.concat([source_a,source_b],axis=0).drop_duplicates(subset='Account_Number',keep=False)

,Transaction_ID,Account_Number,Date,Amount
3,TXN004,12348,2023-01-04,300.0
2,TXN007,12349,2023-01-05,100.0


Solution C

In [99]:
import numpy as np
diff = np.setxor1d(np.array(source_a.Account_Number), np.array(source_b.Account_Number))
pd.concat([source_a.loc[source_a['Account_Number'].isin(list(diff)),:],source_b.loc[source_b['Account_Number'].isin(list(diff)),:]])

,Transaction_ID,Account_Number,Date,Amount
3,TXN004,12348,2023-01-04,300.0
2,TXN007,12349,2023-01-05,100.0


Matches

In [63]:
value_discrepancies = reconciled[
    (reconciled['_merge'] == 'both') & 
    (reconciled['Amount_SourceA'] != reconciled['Amount_SourceB'])
]
display(value_discrepancies)


,Transaction_ID_SourceA,Account_Number,Date_SourceA,Amount_SourceA,Transaction_ID_SourceB,Date_SourceB,Amount_SourceB,_merge
2,TXN003,12347,2023-01-03,750.0,TXN008,2023-01-03,700.0,both


Reconciliation report

In [64]:
print(f"Total Transactions in Source A: {len(source_a)}")
print(f"Total Transactions in Source B: {len(source_b)}")
print(f"Mismatched Transactions: {len(mismatches)}")
print(f"Value Discrepancies: {len(value_discrepancies)}")


Total Transactions in Source A: 4
Total Transactions in Source B: 4
Mismatched Transactions: 2
Value Discrepancies: 1


Example 2 : Combine two DataFrame objects by filling null values in one DataFrame with non-null values from other DataFrame

In [105]:
import pandas as pd

source_a = pd.DataFrame({
    'Transaction_ID': ['TXN001', 'TXN002', 'TXN003', 'TXN004'],
    'Account_Number': [12345, 12346, 12347, 12348],
    'Date': [np.nan, '2023-01-02', '2023-01-03', '2023-01-04'],
    'Amount': [500.00, 1500.00, np.nan, 300.00]
})

source_b = pd.DataFrame({
    'Transaction_ID': ['TXN005', 'TXN006', 'TXN007', 'TXN008'],
    'Account_Number': [12345, 12346, 12349, 12347],
    'Date': ['2023-01-01', '2023-01-02', np.nan, '2023-01-03'],
    'Amount': [500.00, np.nan, 100.00, 300.00]
})

display(source_a)
display(source_b)

,Transaction_ID,Account_Number,Date,Amount
0,TXN001,12345,NaN,500.0
1,TXN002,12346,2023-01-02,1500.0
2,TXN003,12347,2023-01-03,NaN
3,TXN004,12348,2023-01-04,300.0


,Transaction_ID,Account_Number,Date,Amount
0,TXN005,12345,2023-01-01,500.0
1,TXN006,12346,2023-01-02,NaN
2,TXN007,12349,NaN,100.0
3,TXN008,12347,2023-01-03,300.0


In [106]:
source_a.combine_first(source_b,)

,Transaction_ID,Account_Number,Date,Amount
0,TXN001,12345,2023-01-01,500.0
1,TXN002,12346,2023-01-02,1500.0
2,TXN003,12347,2023-01-03,100.0
3,TXN004,12348,2023-01-04,300.0
